# Neural Network, Long Short Term Memory model.  
# Multi-variate input and multi-step output for series forecasting.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


# Datasets used for this project are. Historical prices of Soy in Argentina (Rosario market). Inflation in USA (impacts in dolar values all over the world). Oil Prices (Brent and WTI). 

## I will merge all the datasets and pair them with the main soy price dataset which has records since 2004.
### Special focus on dates as they will be important to merge datasets.

In [74]:
dataset = pd.read_csv('./datasetRofex2.csv', parse_dates=True)
inflation = pd.read_csv('./inflacionusa.csv', parse_dates=True, names=['year', 'month', 'inflation'])
brent = pd.read_csv('./brent-daily.csv', parse_dates=True)
wti = pd.read_csv('./wti-daily.csv', parse_dates=True)

In [75]:
dataset.head()

,Id,Fecha,Open,High,Low,Last,Cierre,Aj.Dif.,Mon.,OI.Vol.,OI.Dif.,Vol.Ope.,Unidad,DolarB.N.,DolarItau.,Dif.Sem
0,1,05/01/2004 12:00:00 a.m.,0.0,0.0,0.0,0.0,221.0,13.0,D,0.0,0.0,0,TONS,2.920,2.905,-221.0
1,2,06/01/2004 12:00:00 a.m.,0.0,0.0,0.0,0.0,221.0,0.0,D,0.0,0.0,0,TONS,2.880,2.870,-221.0
2,3,07/01/2004 12:00:00 a.m.,0.0,0.0,0.0,0.0,219.0,-2.0,D,0.0,0.0,0,TONS,2.865,2.882,-219.0
3,4,08/01/2004 12:00:00 a.m.,0.0,0.0,0.0,0.0,220.0,1.0,D,0.0,0.0,0,TONS,2.887,2.884,-220.0
4,5,09/01/2004 12:00:00 a.m.,0.0,0.0,0.0,0.0,220.0,0.0,D,0.0,0.0,0,TONS,2.892,2.900,-220.0


In [76]:
inflation.head()

,year,month,inflation
0,2004,1,1.9
1,2004,2,1.7
2,2004,3,1.7
3,2004,4,2.3
4,2004,5,3.1


In [77]:
brent.head()

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


In [78]:
wti.head()

,Date,Price
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87


In [79]:
dataset['Date'] = pd.to_datetime(dataset['Fecha'], dayfirst=True)
brent['Date'] = pd.to_datetime(brent['Date'], yearfirst=True)
wti['Date'] = pd.to_datetime(wti['Date'],yearfirst=True)

In [80]:
brent['year'] = brent['Date'].dt.year
wti['year'] = wti['Date'].dt.year
dataset['Year'] = dataset['Date'].dt.year
dataset['month'] = dataset['Date'].dt.month

In [81]:
mask_brent = brent['year']<2004
mask_wti = wti['year']<2004

In [82]:
brent = brent.loc[~mask_brent,:].reset_index().drop(['index'], axis=1)
wti = wti.loc[~mask_wti,:].reset_index().drop(['index'], axis=1)

In [83]:
df = pd.merge(dataset, brent, how='left', on='Date')

In [84]:
df = pd.merge(df, wti, how='left', on='Date')

In [85]:
df.drop(['Fecha', 'year_x', 'year_y', 'Unidad', 'Dif.Sem', 'Open', 'High', 'Low', 'Last', 'OI.Vol.', 'OI.Dif.', 'Vol.Ope.', 'Mon.', 'Aj.Dif.'], axis=1, inplace=True)

In [86]:
df.head()

,Id,Cierre,DolarB.N.,DolarItau.,Date,Year,month,Price_x,Price_y
0,1,221.0,2.920,2.905,2004-01-05,2004,1,32.30,33.71
1,2,221.0,2.880,2.870,2004-01-06,2004,1,31.20,33.54
2,3,219.0,2.865,2.882,2004-01-07,2004,1,30.99,33.57
3,4,220.0,2.887,2.884,2004-01-08,2004,1,31.11,34.27
4,5,220.0,2.892,2.900,2004-01-09,2004,1,31.91,34.38


In [87]:
df['ik'] = df['Year'].astype(str)+'-'+df['month'].astype(str)

In [88]:
inflation['ik'] = inflation['year'].astype(str)+'-'+inflation['month'].astype(str)

In [89]:
df = pd.merge(df, inflation, how='left', on='ik')

In [94]:
df.rename(columns={'inflation': 'USA_Inflation', 'Cierre': 'Closing', 'DolarB.N.':'Dolar_BN', 'DolarItau.':'Dolar_Itau', 'Price_x':'Brent', 'Price_y':'WTI'}, inplace=True)

In [95]:
cols = ['Id', 'Date', 'Dolar_BN', 'Dolar_Itau', 'Brent', 'WTI', 'USA_Inflation', 'Closing']

In [99]:
df = df[cols]

In [100]:
df.isna().sum()

Id                 0
Date               0
Dolar_BN           0
Dolar_Itau         7
Brent             70
WTI              102
USA_Inflation      0
Closing            0
dtype: int64

## The dataset has some nan values I'd like to fill with the previous value.

In [104]:
df.fillna(method='ffill', inplace=True)
df.isna().sum()

Id               0
Date             0
Dolar_BN         0
Dolar_Itau       0
Brent            0
WTI              0
USA_Inflation    0
Closing          0
dtype: int64

## Without nan values the dataset is completed. I'm going to save it as a csv for the modeling phase.

In [105]:
df.to_csv('./dataset.csv', index=False, header=True)